## 신경망 구축
신경망은 데이터에 대한 작업을 수행하는 계층/모듈로 구성됩니다. `torch.nn` 네임스페이스는 자체 신경망을 구축하는데 필요한 모든 빌딩 블록을 제공합니다. Pytorch의 모든 모듈은 `nn.module` 하위 클래스 입니다. 신경망은 다른 모듈(계층)로 구성된 모듈 자체입니다. 이 중첩 구조를 통해 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있습니다.

다음 섹션에서는 `FashionMNIST` 데이터 세트에서 이미지를 분류하기 위해 신경망을 구축합니다.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

C:\Users\LUDOBICO\AppData\Roaming\Python\Python37\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 교육용 장치 가져오기
가능할 경우 `GPU`와 같은 하드웨어 가속기에서 모델을 훈련할 수 있기를 원합니다. `torch.cuda`를 사용할 수 있는지 확인하고 그렇지 않으면 `CPU`를 계속 사용합니다.

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


## 모델 레이어
`FashionMNIST` 모델의 레이어를 분석해 봅시다. 이를 설명하기 위해 28*28 크기의 3개 이미지로 구성된 샘플 미니 배치를 가져와 네트워크를 통과할때 어떤 일이 발생하는지 확인합니다.

In [7]:
input_image = torch.rand(3,28,28)
print(input_image)
print(input_image.size())

tensor([[[0.6658, 0.8941, 0.8231,  ..., 0.9084, 0.6256, 0.7452],
         [0.1455, 0.5973, 0.4038,  ..., 0.6074, 0.8778, 0.4367],
         [0.3683, 0.2715, 0.9017,  ..., 0.6207, 0.8422, 0.0907],
         ...,
         [0.3339, 0.8541, 0.2306,  ..., 0.6898, 0.5091, 0.4648],
         [0.1584, 0.3947, 0.7571,  ..., 0.5516, 0.0486, 0.4440],
         [0.4284, 0.9136, 0.0492,  ..., 0.5379, 0.3886, 0.8503]],

        [[0.9301, 0.7293, 0.5883,  ..., 0.7988, 0.2654, 0.9554],
         [0.5976, 0.1272, 0.1164,  ..., 0.6664, 0.1527, 0.6922],
         [0.2858, 0.0706, 0.5293,  ..., 0.8955, 0.3994, 0.3778],
         ...,
         [0.2582, 0.8723, 0.9906,  ..., 0.3431, 0.0838, 0.7146],
         [0.3864, 0.9524, 0.7115,  ..., 0.4412, 0.8238, 0.7761],
         [0.8152, 0.2624, 0.1337,  ..., 0.0736, 0.8826, 0.2660]],

        [[0.7115, 0.3883, 0.1458,  ..., 0.1956, 0.8849, 0.4227],
         [0.4751, 0.9977, 0.5479,  ..., 0.3873, 0.4514, 0.0686],
         [0.7392, 0.9315, 0.5417,  ..., 0.8042, 0.8476, 0.

## nn.Flatten*
`nn.Flatten` 레이어를 초기화하여 각 2D 28*28 이미지를 784 픽셀 값의 연속 배열로 변환합니다. (미니배치 차원(dim=0에서)이 유지됨)

`Flatten`은 일반적으로 입력 데이터의 구조를 깊은 계층 구조에서 평탄한 구조로 바꾸는 것을 의미합니다. 이것은 주로 신경망의 입력층과 은닉층사이에서 수행됩니다. 예를 들어, 이미지 데이터의 경우 깊은 계층 구조가 일반적으로 각 계층에서 고차원의 출력을 생성하기 때문에 이를 평탄한 구조로 바꾸기 위해 사용할 수 있습니다. 이렇게 하면 일반적으로 입력 데이터의 구조차원 구조가 일어나기 때문에 신경망이 처리하기 어려운 고차원 데이터를 처리할 수 있는 저차원 데이터로 바꾸어 주기 때문입니다.

In [6]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image)
print(flat_image.size())

tensor([[0.7131, 0.3762, 0.1229,  ..., 0.6660, 0.6250, 0.0683],
        [0.8084, 0.1210, 0.0769,  ..., 0.0334, 0.6827, 0.0660],
        [0.9019, 0.1220, 0.4534,  ..., 0.8527, 0.1880, 0.0622]])
torch.Size([3, 784])


## nn.linear
`선형 계층`은 저장된 가중치와 편향을 사용하여 입력에 선형 변환을 적용하는 모듈입니다.

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1)
print(hidden1.size())

tensor([[ 0.2474,  0.4527, -0.6020, -0.1155, -0.3309, -0.1676, -0.2441, -0.9399,
         -0.6062, -0.2157,  0.5055,  0.6485, -0.1223, -0.1734,  0.2810,  0.0141,
         -0.0927, -0.2233,  0.3967,  0.2691],
        [ 0.3556,  0.5093, -0.4995, -0.2223, -0.0264, -0.4979, -0.0013, -0.4729,
         -0.2557, -0.3703,  0.1600,  0.9676, -0.0381, -0.0076, -0.1474, -0.1116,
         -0.1684,  0.2044,  0.5976,  0.1881],
        [ 0.3988,  0.5375, -0.9643, -0.2940, -0.1489, -0.3664, -0.0280, -0.5056,
         -0.3783, -0.2094,  0.2640,  0.5699, -0.1173, -0.1478,  0.1039,  0.1238,
         -0.2431,  0.0263,  0.2342,  0.5383]], grad_fn=<AddmmBackward0>)
torch.Size([3, 20])


## nn.ReLU
비선형 활성화는 모델의 입력과 출력 사이에 복잡한 매핑을 만드는 것입니다. 선형 변환 후에 적용되어 *비선형성* 을 도입하여 신경망이 다양한 현상을 하습하도록 돕습니다.

이 모델에서는 선형 레이어 간에 `nn.ReLU`를 사용 하지만 모델에 비선형성을 도입하기 위한 다른 활성화가 있습니다.

In [9]:
print('Before ReLU: {0}'.format(hidden1))
hidden1 = nn.ReLU()(hidden1)
print('After ReLU: {0}'.format(hidden1))

Before ReLU: tensor([[ 0.2474,  0.4527, -0.6020, -0.1155, -0.3309, -0.1676, -0.2441, -0.9399,
         -0.6062, -0.2157,  0.5055,  0.6485, -0.1223, -0.1734,  0.2810,  0.0141,
         -0.0927, -0.2233,  0.3967,  0.2691],
        [ 0.3556,  0.5093, -0.4995, -0.2223, -0.0264, -0.4979, -0.0013, -0.4729,
         -0.2557, -0.3703,  0.1600,  0.9676, -0.0381, -0.0076, -0.1474, -0.1116,
         -0.1684,  0.2044,  0.5976,  0.1881],
        [ 0.3988,  0.5375, -0.9643, -0.2940, -0.1489, -0.3664, -0.0280, -0.5056,
         -0.3783, -0.2094,  0.2640,  0.5699, -0.1173, -0.1478,  0.1039,  0.1238,
         -0.2431,  0.0263,  0.2342,  0.5383]], grad_fn=<AddmmBackward0>)
After ReLU: tensor([[0.2474, 0.4527, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.5055, 0.6485, 0.0000, 0.0000, 0.2810, 0.0141, 0.0000, 0.0000,
         0.3967, 0.2691],
        [0.3556, 0.5093, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.1600, 0.9676, 0.0000, 0.0000, 0.0000

## nn.Sequential

`nn.Sequential`은 순서가 지정된 모듈 컨테이너입니다. 데이터는 정의된 것과 동일한 순서로 모든 모듈을 통해 전달됩니다. 순차 컨테이너를 사용합니다.

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)

input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
print(logits)

tensor([[ 0.2127,  0.0380,  0.1381, -0.2528, -0.1236,  0.0436,  0.0030,  0.2921,
          0.0024, -0.1352],
        [ 0.2580, -0.0098,  0.1397, -0.3172, -0.0773,  0.1411,  0.1899,  0.3030,
          0.0407, -0.0591],
        [ 0.2247,  0.0909,  0.1360, -0.2133,  0.0224,  0.1266,  0.0388,  0.1836,
          0.0056, -0.0943]], grad_fn=<AddmmBackward0>)


## nn.Softmax
신경망의 마지막 선형 계층은 nn.Softmax 모듈로 전달되는 [-infty, infty]의 원시 값인 logits을 반환합니다. 로짓은 각 클래스에 대한 모델의 예측 확률을 나타내는 값[0,1]로 조정됩니다. 매개변수는 값의 합이 1이 되어야 하는 차원을 나타냅니다.

In [13]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.1196, 0.1004, 0.1110, 0.0751, 0.0854, 0.1010, 0.0969, 0.1294, 0.0969,
         0.0844],
        [0.1200, 0.0918, 0.1066, 0.0675, 0.0858, 0.1068, 0.1121, 0.1255, 0.0966,
         0.0874],
        [0.1179, 0.1032, 0.1079, 0.0761, 0.0963, 0.1069, 0.0979, 0.1132, 0.0947,
         0.0857]], grad_fn=<SoftmaxBackward0>)
